In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
df = pd.read_excel("./data/iris_xlsx.xlsx")
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [5]:
x = df.drop(["Species"], axis=1) # 주성분 분석에서 범주형 변수는 조심해야한다
x = StandardScaler().fit_transform(x)
x[:3]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ]])

In [9]:
df.columns[:-1]

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], dtype='object')

In [10]:
x = pd.DataFrame(x, columns = df.columns[:-1])

In [11]:
x.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444


In [17]:
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [18]:
pc = pca.fit_transform(x)
pc[:3]

array([[-2.26470281,  0.4800266 , -0.12770602],
       [-2.08096115, -0.67413356, -0.23460885],
       [-2.36422905, -0.34190802,  0.04420148]])

In [19]:
df_pc = pd.DataFrame(data = pc, columns = ["pc1", "pc2", "pc3"])
df_pc.head(2)

,pc1,pc2,pc3
0,-2.264703,0.480027,-0.127706
1,-2.080961,-0.674134,-0.234609


In [20]:
pca.explained_variance_ratio_

array([0.72962445, 0.22850762, 0.03668922])

In [21]:
pca.explained_variance_ratio_.sum()

0.9948212908928452

In [22]:
pca.explained_variance_ratio_[:2].sum()

0.9581320720000164

In [23]:
import statsmodels.formula.api as smf

In [24]:
bike = pd.read_csv("./data/bike.csv")

In [26]:
model = smf.ols("casual ~ weather + temp + atemp + humidity",
               data = bike)
model

In [27]:
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 casual   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     1296.
Date:                Tue, 12 Oct 2021   Prob (F-statistic):               0.00
Time:                        17:22:06   Log-Likelihood:                -55903.
No. Observations:               10886   AIC:                         1.118e+05
Df Residuals:                   10881   BIC:                         1.119e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     25.6758      1.829     14.039      0.000      22.091      29.261
weather        1.9962      0.682      2.926      0.003       0.659       3.333
temp           0.9858      0.296      3.335      0.001       0.406       1.565
atemp          1.7535      0.272      6.459      0.000       1.221       2.286
humidity      -0.8711      0.023    -38.475      0.000      -0.915      -0.827
==============================================================================
Omnibus:                     5695.737   Durbin-Watson:                   0.172
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            46302.172
Skew:                           2.398   Prob(JB):                         0.00
Kurtosis:                      11.893   Cond. No.                         334.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
bike_pc = bike.loc[:, "weather":"humidity"]
bike_pc.head(2)

,weather,temp,atemp,humidity
0,1,9.84,14.395,81
1,1,9.02,13.635,80


In [29]:
x = StandardScaler().fit_transform(bike_pc)
x = pd.DataFrame(x, columns=bike_pc.columns)
pca = PCA(n_components=3)
pc = pca.fit_transform(x)
df_pc = pd.DataFrame(data = pc, columns = ["pc1", "pc2", "pc3"])
df_pc.head(2)

,pc1,pc2,pc3
0,-1.729483,-0.075251,-1.173271
1,-1.858499,-0.136112,-1.136852


In [30]:
df_pc["casual"] = bike["casual"]

In [31]:
model = smf.ols("casual ~ pc1 + pc2 + pc3",
               data = df_pc)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 casual   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     1726.
Date:                Tue, 12 Oct 2021   Prob (F-statistic):               0.00
Time:                        17:25:40   Log-Likelihood:                -55905.
No. Observations:               10886   AIC:                         1.118e+05
Df Residuals:                   10882   BIC:                         1.118e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.0220      0.394     91.379      0.000      35.249      36.795
pc1           17.6358      0.278     63.347      0.000      17.090      18.181
pc2           -7.8790      0.335    -23.533      0.000      -8.535      -7.223
pc3           12.6580      0.511     24.747      0.000      11.655      13.661
==============================================================================
Omnibus:                     5707.860   Durbin-Watson:                   0.172
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            46569.247
Skew:                           2.403   Prob(JB):                         0.00
Kurtosis:                      11.920   Cond. No.                         1.84
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
bike.corr()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
season,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.096758,0.164011,0.163439
holiday,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,0.043799,-0.020956,-0.005393
workingday,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,-0.319111,0.119460,0.011594
weather,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.135918,-0.109340,-0.128655
temp,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.467097,0.318571,0.394454
atemp,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.462067,0.314635,0.389784
humidity,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.348187,-0.265458,-0.317371
windspeed,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,0.092276,0.091052,0.101369
casual,0.096758,0.043799,-0.319111,-0.135918,0.467097,0.462067,-0.348187,0.092276,1.000000,0.497250,0.690414
registered,0.164011,-0.020956,0.119460,-0.109340,0.318571,0.314635,-0.265458,0.091052,0.497250,1.000000,0.970948


In [34]:
df_pc.corr()

,pc1,pc2,pc3,casual
pc1,1.000000e+00,9.626152e-17,1.607082e-16,0.499850
pc2,9.626152e-17,1.000000e+00,-2.281542e-17,-0.185692
pc3,1.607082e-16,-2.281542e-17,1.000000e+00,0.195271
casual,4.998504e-01,-1.856925e-01,1.952714e-01,1.000000
